In [1]:
# Import dependencies
import pandas as pd
import glob


In [2]:
# Create an empty list for the dataframes and create path
dfs = []

# This makes an array of paths in this directory all ending in .csv
csv_files = glob.glob('*.csv')
csv_files

['202107-citibike-tripdata.csv',
 '202207-citbike-tripdata.csv',
 '202307-citibike-tripdata.csv']

In [3]:
# Read CSV files and store in list
count = 0
for file in csv_files:
    print(f'Currently Extracting from {file}...')
    df = pd.read_csv(file, dtype={'ride_id':str, 'rideable_type':str, 'started_at':str, 'ended_at':str,
        'start_station_name':str, 'start_station_id':str, 'end_station_name':str,
        'end_station_id':str, 'start_lat':float, 'start_lng':float, 'end_lat':float, 'end_lng':float,
        'member_casual':str})
    
    # Remove rows with null data
    clean_df = df.dropna()
    
    # Append dataframe
    count += 1
    print(f'Done extracting {count} file/s')
    dfs.append(clean_df)

Currently Extracting from 202107-citibike-tripdata.csv...
Done extracting 1 file/s
Currently Extracting from 202207-citbike-tripdata.csv...
Done extracting 2 file/s
Currently Extracting from 202307-citibike-tripdata.csv...
Done extracting 3 file/s


In [4]:
# Concatenate dataframes
combined_df = pd.concat(dfs, ignore_index=True)

In [5]:
# Select only colummns needed
combined_df = combined_df[['ride_id','rideable_type','started_at','ended_at','start_station_name','end_station_name','start_lat','start_lng','end_lat','end_lng','member_casual']]
combined_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,start_lat,start_lng,end_lat,end_lng,member_casual
0,A14B340FDE7576EA,electric_bike,2021-07-02 16:57:23,2021-07-02 17:09:47,W 90 St & Amsterdam Ave,E 88 St & 1 Ave,40.790179,-73.972889,40.778301,-73.948813,casual
1,1726632C7F033C36,electric_bike,2021-07-10 07:40:55,2021-07-10 07:58:10,W 31 St & 7 Ave,Allen St & Rivington St,40.749156,-73.991600,40.720196,-73.989978,casual
2,3B287B65EE038E8B,electric_bike,2021-07-09 13:25:58,2021-07-09 13:30:13,W 169 St & Fort Washington Ave,Wadsworth Ave & W 175 St,40.842842,-73.942125,40.846052,-73.937648,member
3,DC9F7508954A9134,classic_bike,2021-07-09 12:45:36,2021-07-09 12:59:54,Leonard St & Church St,W 15 St & 6 Ave,40.717571,-74.005549,40.738046,-73.996430,member
4,B8B799986A18C285,classic_bike,2021-07-29 19:28:11,2021-07-29 19:52:09,Market St & Cherry St,FDR Drive & E 35 St,40.710762,-73.994004,40.744219,-73.971212,casual


In [6]:
# Make addressess have similar coordinates by averaging them
grouped = combined_df.groupby('start_station_name')['start_lng'].mean().reset_index()
combined_df = combined_df.merge(grouped, on='start_station_name', how='left', suffixes=('', '_avg'))
combined_df['start_lng'] = combined_df['start_lng_avg']
combined_df.drop(columns=['start_lng_avg'], inplace=True)

grouped = combined_df.groupby('start_station_name')['start_lat'].mean().reset_index()
combined_df = combined_df.merge(grouped, on='start_station_name', how='left', suffixes=('', '_avg'))
combined_df['start_lat'] = combined_df['start_lat_avg']
combined_df.drop(columns=['start_lat_avg'], inplace=True)

grouped = combined_df.groupby('end_station_name')['end_lng'].mean().reset_index()
combined_df = combined_df.merge(grouped, on='end_station_name', how='left', suffixes=('', '_avg'))
combined_df['end_lng'] = combined_df['end_lng_avg']
combined_df.drop(columns=['end_lng_avg'], inplace=True)

grouped = combined_df.groupby('end_station_name')['end_lat'].mean().reset_index()
combined_df = combined_df.merge(grouped, on='end_station_name', how='left', suffixes=('', '_avg'))
combined_df['end_lat'] = combined_df['end_lat_avg']
combined_df.drop(columns=['end_lat_avg'], inplace=True)


In [ ]:
# Round Latitude and Longitude to 2 decimal places
# combined_df['start_lat'] = combined_df['start_lat'].round(2)
# combined_df['start_lng'] = combined_df['start_lng'].round(2)
# combined_df['end_lat'] = combined_df['end_lat'].round(2)
# combined_df['end_lng'] = combined_df['end_lng'].round(2)

In [ ]:
combined_df.to_csv('output/final_output.csv',index = False)